In [11]:
#IMPORTS
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

import json
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
bucket_name = '340915149wiki' 
client = storage.Client()
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
#CHANGE THIS
# wiki1000_body = parquetFile.select("id", "text").rdd
wiki1000_title = parquetFile.select("id", "title").rdd
# wiki1000_anchor = parquetFile.select("id", "anchor_text").rdd

ID TO TITLES

In [9]:
ids_titles = wiki1000_title.collectAsMap()

In [7]:
with open('titles.json', 'w') as titles:
     json.dump(ids_titles, titles)

titles_src = "titles.json"
titles_dst = f'gs://{bucket_name}/titles/{titles_src}'
!gsutil cp $titles_src $titles_dst

Copying file://titles.json [Content-Type=application/json]...
/ [1 files][ 33.5 KiB/ 33.5 KiB]                                                
Operation completed over 1 objects/33.5 KiB.                                     


In [13]:
%cd -q /home/dataproc
!ls inverted_index.py
sc.addFile("/home/dataproc/inverted_index.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index import InvertedIndex

inverted_index.py


In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
ps = PorterStemmer()

In [6]:
def get_tf(id, text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # filtered tokens
    tokens = [token for token in tokens if token not in all_stopwords]
    # use counter to count and map values with id 
    return list(map(lambda x: (x[0], (id, x[1])), list(Counter(tokens).items())))

In [7]:
def get_DL(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # filtered tokens
    tokens = [token for token in tokens if token not in all_stopwords]
    return len(tokens)

BODY

In [8]:
#get tfs
body_tfs = wiki1000_body.flatMap(lambda x: get_tf(x[0], x[1]))

In [9]:
#sort pls
body_postings = body_tfs.groupByKey().mapValues(InvertedIndex.reduce_word_counts)
body_postings_filtered = body_postings.filter(lambda x: len(x[1])>50)

In [10]:
body_df = InvertedIndex.calculate_df(body_postings_filtered)
body_df_dict = body_df.collectAsMap()

In [11]:
body_DL = wiki1000_body.map(lambda x: (x[0], get_DL(x[1]))).collectAsMap()

In [ ]:
body_tot_term = body_postings_filtered.mapValues(InvertedIndex.get_total_term).collectAsMap()

In [13]:
body_posting_locs_list = InvertedIndex.partition_postings_and_write(body_postings_filtered, bucket_name, "body").collect()

In [14]:
body_super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            body_super_posting_locs[k].extend(v)

In [15]:
# Create inverted index instance
body_inv_index = InvertedIndex()
body_inv_index.DL = body_DL
# Adding the posting locations dictionary to the inverted index
body_inv_index.posting_locs = body_super_posting_locs
# Add the token - df dictionary to the inverted index
body_inv_index.df = body_df_dict
body_inv_index.term_total = body_tot_term
# write the global stats out
body_inv_index.write_index('.', 'body_index')
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/body/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 69.0 MiB/ 69.0 MiB]                                                
Operation completed over 1 objects/69.0 MiB.                                     


In [ ]:
# body_inv_index.read_posting_list('state', 'body', bucket_name)

TITLE

In [38]:
def get_title_tf(id, text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # filtered tokens
    tokens = [token for token in tokens if token not in all_stopwords]
    # use counter to count and map values with id 
    stems = []
    for token in tokens:
        stems.append(ps.stem(token))
    return list(map(lambda x: (x[0], (id, x[1])), list(Counter(stems).items())))

def get_title_DL(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # filtered tokens
    tokens = [token for token in tokens if token not in all_stopwords]
    stems = []
    for token in tokens:
        stems.append(ps.stem(token))
    return len(stems) # TODO Maybe not filtered


In [39]:
#get tfs
title_tfs = wiki1000_title.flatMap(lambda x: get_title_tf(x[0], x[1]))

In [40]:
# #sort pls
title_postings = title_tfs.groupByKey().mapValues(InvertedIndex.reduce_word_counts)

In [41]:
title_df = InvertedIndex.calculate_df(title_postings)
title_df_dict = title_df.collectAsMap()

In [42]:
title_DL = wiki1000_title.map(lambda x: (x[0], get_title_DL(x[1]))).collectAsMap()

In [43]:
title_tot_term = title_tfs.groupByKey().mapValues(InvertedIndex.get_total_term).collectAsMap()

In [44]:
title_posting_locs_list = InvertedIndex.partition_postings_and_write(title_postings, bucket_name, "title").collect()

In [45]:
title_super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            title_super_posting_locs[k].extend(v)

In [46]:
# Create inverted index instance
title_inv_index = InvertedIndex()
title_inv_index.DL = title_DL
# Adding the posting locations dictionary to the inverted index
title_inv_index.posting_locs = title_super_posting_locs
# Add the token - df dictionary to the inverted index
title_inv_index.df = title_df_dict
title_inv_index.term_total = title_tot_term
# write the global stats out
title_inv_index.write_index('.', 'title_index')
index_src2 = "title_index.pkl"
index_dst2 = f'gs://{bucket_name}/title/{index_src2}'
!gsutil cp $index_src2 $index_dst2

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
| [1 files][126.1 MiB/126.1 MiB]                                                
Operation completed over 1 objects/126.1 MiB.                                    


In [47]:
title_inv_index.read_posting_list('anarchism', 'title', bucket_name)

[]

In [ ]:
title_inv_index.read_posting_list('anarch', 'title', bucket_name)

ANCHOR

In [20]:
def get_anchor_tf(id, rows):
    rows2 = list(map(lambda x: x[1], rows))
    text = ''
    for t in rows2:
        text+=t+" "
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # filtered tokens
    tokens = [token for token in tokens if token not in all_stopwords]
    # use counter to count and map values with id 
    return list(map(lambda x: (x[0], (id, x[1])), list(Counter(tokens).items())))

In [21]:
def get_anchor_DL(rows):
    rows2 = list(map(lambda x: x[1], rows))
    text = ''
    for t in rows2:
        text+=t+" "
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # filtered tokens
    tokens = [token for token in tokens if token not in all_stopwords]
    return len(tokens) # TODO Maybe not filtered

In [22]:
#get tfs
anchor_tfs = wiki1000_anchor.flatMap(lambda x: get_anchor_tf(x[0], x[1]))

In [23]:
# sort pls
anchor_postings = anchor_tfs.groupByKey().mapValues(InvertedIndex.reduce_word_counts)
anchor_postings_filtered = body_postings.filter(lambda x: len(x[1])>20)

In [24]:
anchor_df = InvertedIndex.calculate_df(anchor_postings_filtered)
anchor_df_dict = anchor_df.collectAsMap()

In [25]:
anchor_DL = wiki1000_anchor.map(lambda x: (x[0], get_anchor_DL(x[1]))).collectAsMap()

In [26]:
anchor_tot_term = anchor_postings_filtered.mapValues(InvertedIndex.get_total_term).collectAsMap()

In [27]:
anchor_posting_locs_list = InvertedIndex.partition_postings_and_write(anchor_postings_filtered, bucket_name, "anchor").collect()

In [28]:
anchor_super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            anchor_super_posting_locs[k].extend(v)

In [30]:
# Create inverted index instance
anchor_inv_index = InvertedIndex()
anchor_inv_index.DL = anchor_DL
# Adding the posting locations dictionary to the inverted index
anchor_inv_index.posting_locs = anchor_super_posting_locs
# Add the token - df dictionary to the inverted index
anchor_inv_index.df = anchor_df_dict
anchor_inv_index.term_total = anchor_tot_term
# write the global stats out
anchor_inv_index.write_index('.', 'anchor_index')
index_src3 = "anchor_index.pkl"
index_dst3 = f'gs://{bucket_name}/anchor/{index_src3}'
!gsutil cp $index_src3 $index_dst3

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 91.4 MiB/ 91.4 MiB]                                                
Operation completed over 1 objects/91.4 MiB.                                     


In [ ]:
# anchor_inv_index.read_posting_list('teresa', 'anchor', bucket_name)

PAGE RANK

In [ ]:
def get_ids_from_anchors(id,anchorlist):
    ids = []
    for anchor in anchorlist:
        ids.append((id, anchor[0]))
    return ids
  
def generate_graph(pages):
    verticesFromLinks = pages.flatMap(lambda x: x[1]).map(lambda x: x[0])
    verticeFromIds = pages.map(lambda x: x[0])
    vertices = verticeFromIds.union(verticesFromLinks).distinct().map(lambda x: (x, ))
  
    edges = pages.flatMap( lambda x: (get_ids_from_anchors(x[0],x[1]))).distinct()

    return edges, vertices

In [ ]:
edges, vertices = generate_graph(wiki1000_anchor)
v_cnt, e_cnt = vertices.count(), edges.count()

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
# pr = pr.sort(col('pagerank').desc())
# pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
# pr.show()

# dictpr - dictionary: (key: doc_id, value: doc_pagerank)
dictpr = {}
for row in pr.toPandas().iterrows():
    dictpr[int(row[1][0])]=row[1][1]

with open('pr.json', 'w') as pr:
     json.dump(dictpr, pr)

pr_src = "pr.json"
pr_dst = f'gs://{bucket_name}/pr/{pr_src}'
!gsutil cp $pr_src $pr_dst

PAGE VIEWS

In [ ]:
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
!wget -N $pv_path
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})

In [ ]:
with open('pv.json', 'w') as pv:
     json.dump(wid2pv, pv)

pv_src = "pv.json"
pv_dst = f'gs://{bucket_name}/pv/{pv_src}'
!gsutil cp $pv_src $pv_dst